# Sherdog fights scraper


In [1]:
import requests as req
from bs4 import BeautifulSoup # this module helps in web scrapping
import pandas as pd
import numpy as np
import random
from unidecode import unidecode # this module helps in search query
import os 
import glob 
from tqdm import tqdm
from random import randint, uniform
from time import sleep

In [2]:
def search(fighter):
    url = 'https://www.google.com/search?q=' + unidecode(fighter) + " sherdog" # search query
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"}
    data = req.get(url, headers = headers).text
    soup = BeautifulSoup(data,"html.parser")
    return soup.h3.parent["href"]

In [3]:
def get_website(fighter):
    url = search(fighter)
    user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1',
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363',
    ]
    headers={"User-Agent": user_agent_list[random.randint(0, len(user_agent_list)-1)]}
    data  = req.get(url, headers = headers).text 
    return data, url

In [4]:
def get_record(fighter):
    fighter_data, url = get_website(fighter)
    fighter_soup = BeautifulSoup(fighter_data, "html.parser") 
    # Finds fighter information
    nationality = ""
    if fighter_soup.find("strong", itemprop="nationality") != None:
        nationality = fighter_soup.find("strong", itemprop="nationality").text
    nickname = ""
    if fighter_soup.find("span", class_="nickname") != None:
        nickname = fighter_soup.find("span", class_="nickname").find("em").text
    birthday = ""
    if fighter_soup.find("span", itemprop="birthDate") != None:
        birthday = fighter_soup.find("span", itemprop="birthDate").text
    height_ft = ""
    height_cm = ""
    if fighter_soup.find("b", itemprop="height")!= None:
        height_ft = fighter_soup.find("b", itemprop="height").text
        if fighter_soup.find("b", itemprop="height").nextSibling.nextSibling.nextSibling != None:
            height_cm = fighter_soup.find("b", itemprop="height").nextSibling.nextSibling.nextSibling
    # Finds table with fighters record
    table = fighter_soup.find("table", class_="new_table fighter")
    # Creates DataFrame
    fighter_record = pd.DataFrame(columns=["Fighter", "URL", "Nationality", "Nickname", "Birthday", "Height ft", "Height cm", "Opponent", "Result", "Method", "Round", "Time", "Event", "Date", "Referee"])
    # Fills the DataFrame with fighter record
    for row in table.find_all("tr", class_=""):
        col = row.find_all("td")
        if (col != []):
            result = col[0].text
            opponent = col[1].text
            event = col[2].a.text
            date = col[2].find("span", class_="sub_line").text
            method = col[3].b.text
            referee = col[3].span.text
            rnd = col[4].text
            time = col[5].text
            fighter_record = fighter_record._append({"Fighter":fighter, "URL":url, "Nationality":nationality, "Nickname":nickname, "Birthday":birthday, "Height ft":height_ft, "Height cm":height_cm, "Opponent":opponent, "Result":result, "Method":method, "Round":rnd, "Time":time, "Event":event, "Date":date, "Referee":referee}, ignore_index=True)

    return fighter_record

In [5]:
df = pd.read_csv("bouts_new.csv")
fighters_list = pd.concat([df["Fighter A"], df["Fighter B"]], ignore_index=True)

In [6]:
fighters_list=fighters_list.unique()

In [7]:
fighters_list.shape

(2479,)

In [168]:
arr=fighters_list[2160:]

Some fighters names need to be changed to be found on sherdog

In [169]:
arr[arr == "Ian Machado Garry"] = "Ian Garry"

In [170]:
arr[arr == "Sako Chivitchian"] = "Sako Chivitchyan"

In [268]:
arr[arr == "Razak Al-Hassan"] = "Razak Razor Al-Hassan"

In [171]:
records = pd.DataFrame(columns=["Fighter","URL","Nationality","Nickname","Birthday","Height ft","Height cm","Opponent", "Result", "Method", "Round", "Time", "Event", "Date", "Referee"])
counter = 0
for name in tqdm(arr):
    records = pd.concat([records, get_record(name)])
    counter+=1
    if counter == 90:
        records.to_csv("records.csv", index=False)
        sleep(randint(20,40))
        counter=0
records
records.to_csv("records.csv", index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 319/319 [16:23<00:00,  3.08s/it]


We have to change the names back now

In [282]:
df=pd.read_csv("records.csv")

In [283]:
df.loc[df["Fighter"] == "Ian Garry", "Fighter"] = "Ian Machado Garry"
df.loc[df["Fighter"] == "Sako Chivitchyan", "Fighter"] = "Sako Chivitchian"
df.loc[df["Fighter"] == "Razak Razor Al-Hassan", "Fighter"] = "Razak Al-Hassan"

In [284]:
df["Fighter"].unique().shape

(2479,)

We can see that there is the same number of fighters as when we started, that means we got records for all listed fighters

In [280]:
df.to_csv("records.csv", index=False)